In [ ]:
pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install keras_self_attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=d7f2e16031ea0a8047f06f4d2f5f39ec19cf875418c9a89bd761a6a348fd2d73
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_self_attention


In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Concatenate, Dropout, GlobalMaxPooling1D, concatenate, TimeDistributed, RepeatVector
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from keras.utils import to_categorical

from keras_self_attention import SeqSelfAttention
import torch
import torch.nn as nn


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Set the first GPU as the default device
    tf.config.set_visible_devices(gpus[0], 'GPU')

    # Enable memory growth for the GPU
    tf.config.experimental.set_memory_growth(gpus[0], True)

    # Check that the device is being used
    with tf.device('/GPU:0'):
      print("Using GPU:", gpus[0])
  except RuntimeError as e:
    print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Data loading

In [ ]:
accepted_answers_df = pd.read_csv('Accepted_Answers.csv')
accepted_answers_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
9995,"Spring Security Core authAjax, How do I ignore...",<p>When you make an un-authorized attempt to a...,5628199
9996,Dictionary+Queue Data Structure with Active Re...,<p>I'd start by just modeling the behavior you...,5888734
9997,Extend an existing list in Django settings,<p>You should be able to import form <code>dja...,5916209
9998,Python: How to deal with non-subscriptable obj...,<p>The way to fix it is to use dictionaries pr...,5977165


In [ ]:
scored_answer_df = pd.read_csv('Scored_Answers.csv')
scored_answer_df

,Question,Relevant_Answer,question_id
0,in android how to send and receive special sms...,<p>For this you have to create your custom bro...,29165268
1,Python Salary Program Using Conditionals,<p>You're using full quotes where you should b...,29089187
2,How can I show a spinner in a ViewPager when t...,<p>I would do a check for this inside the <cod...,29169909
3,Android: Horizontal list view on custom adaptor,<blockquote>\n <p>layout whose one item cover...,28878917
4,"Even If Testcase failed It is going in ""onTest...",<p>Function <code>verify</code> won't make a t...,29300800
...,...,...,...
9995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
9996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
9997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
9998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [ ]:
ground_truth_df = pd.concat([accepted_answers_df, scored_answer_df], axis=0, ignore_index=True)
ground_truth_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [ ]:
all_answers_df = pd.read_csv('All_answers.csv')
all_answers_df

,question_id,question_title,answer_id,answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...
...,...,...,...,...
33424,73829515,Big Query first non null value record type,73830169,<p>Consider below option</p>\n<pre><code>selec...
33425,73829809,Change Wordpress colors based on logo main color,73829900,"<p>First of all, you need colors from the imag..."
33426,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...
33427,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...


Preprocessing

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation_and_lower(text):
    text = text.lower()
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word not in stop_words]

def remove_usernames(text: str) -> str:
    """
    Remove usernames starting with '@' from the given text.

    Args:
        text (str): The input text containing usernames.

    Returns:
        str: The text with usernames removed.
    """
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", "", text)
    return cleaned_text

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_usernames(text)
    text = remove_urls(text)
    text = remove_punctuation_and_lower(text)
    text_without_newlines = text.replace('\n', ' ')
    words = remove_stopwords(text)
    return ' '.join(words)


In [ ]:
ground_truth_df['Clean_Relevant_Answer'] = ground_truth_df['Relevant_Answer'].apply(clean_text)
ground_truth_df

,Question,Relevant_Answer,question_id,Clean_Relevant_Answer
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519,dont appear returning anything infobar try con...
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802,perl rescue perl e print packb argv0 011010000...
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188,variables mutated inside asynchronous closure ...
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556,attributestokenidlastclaimdate uint attributes...
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861,conn defined either change connectiontodb conn...
...,...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549,using version 0616 set aggregatefalse configen...
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033,question doesnt really anything core graphics ...
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186,using break instruction possibility often cons...
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275,try using f800 englishus locale prefix format ...


In [ ]:
all_answers_df['clean_answer_body'] = all_answers_df['answer_body'].apply(clean_text)
all_answers_df.head()

,question_id,question_title,answer_id,answer_body,clean_answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...,thats excellent something written memory teste...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...,intending reading large number columns records...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...,querying sql server database version 7 replace...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...,roughly memory since dont code laptop using ol...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...,thats definitely good way happen using databas...


In [ ]:
questions = np.array(ground_truth_df['Question']).astype(str)

all_answers = all_answers_df['clean_answer_body'].values.astype(str)



Tokenization

In [ ]:
len(questions), len(all_answers)

(20000, 33429)

In [ ]:
questions_answers = np.append(questions, all_answers, axis=None)

In [ ]:
questions_answers.shape

(53429,)

In [ ]:
del questions
del all_answers


In [ ]:
## defining constants for thr Tokenizer
num_words = 2000
maxlen = 30

In [ ]:
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(questions_answers)
seqs = tokenizer.texts_to_sequences(questions_answers)
encoder_input_data = pad_sequences(seqs, maxlen, padding='post')

Model building

In [ ]:
# define the model architecture
input_shape = (maxlen,)
embed_dim = 150
latent_dim = 128

In [ ]:
## Encoder Model
input_layer = Input(shape=input_shape, name='Encoder-Input')
embedding_layer = Embedding(num_words, embed_dim, input_length=maxlen, name='Body-Word-Embedding', mask_zero=False)(input_layer)
encoder_layer = Bidirectional(LSTM(latent_dim, activation='relu', name='Encoder-Last-BiLSTM'))(embedding_layer)
encoder_model = Model(inputs=input_layer, outputs=encoder_layer, name='Encoder-Model')
seq2seq_encoder_out = encoder_model(input_layer)

## Decoder Model
decoder_layer = RepeatVector(maxlen)(seq2seq_encoder_out)
decoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, name='Decoder-BiLSTM-before'))(decoder_layer)
decoder_out = Dense(num_words, activation='softmax', name='Final-Output-Dense-before')(decoder_lstm)

## Combination - Autoencoder

seq2seq_Model = Model(input_layer, decoder_out)

In [ ]:
seq2seq_Model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder-Input (InputLayer)  [(None, 30)]              0         
                                                                 
 Encoder-Model (Functional)  (None, 256)               585696    
                                                                 
 repeat_vector (RepeatVecto  (None, 30, 256)           0         
 r)                                                              
                                                                 
 bidirectional_1 (Bidirecti  (None, 30, 256)           394240    
 onal)                                                           
                                                                 
 Final-Output-Dense-before   (None, 30, 2000)          514000    
 (Dense)                                                         
                                                             

In [ ]:
# compile model

seq2seq_Model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training model

In [ ]:
### creating a folder in current directory for saving model weights

folder_path ='./Autoencoder_BiLSTM'

if os.path.exists(folder_path) == True:
    None
else:
    os.makedirs(folder_path)

In [ ]:
# ## saving weights

checkpoint_path = './Autoencoder_BiLSTM/weight_{epoch:02d}.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='max',
                                                monitor='val_accuracy')
LR = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_accuracy", factor=0.1, patience=2)
early = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2,restore_best_weights=True)

checkpt_list = [checkpoint, LR, early]

In [ ]:
## training
seq2seq_Model.fit(encoder_input_data, np.expand_dims(encoder_input_data, -1), epochs=10, batch_size=32, validation_split=0.2, callbacks=checkpt_list)

Epoch 1/10
1336/1336 [==============================] - 218s 154ms/step - loss: 3.2614 - accuracy: 0.5323 - val_loss: 4.7272 - val_accuracy: 0.2968 - lr: 0.0010
Epoch 2/10
1336/1336 [==============================] - 172s 129ms/step - loss: 2.9549 - accuracy: 0.5455 - val_loss: 4.4477 - val_accuracy: 0.3118 - lr: 0.0010
Epoch 3/10
1336/1336 [==============================] - 215s 161ms/step - loss: 2.7099 - accuracy: 0.5620 - val_loss: 4.1586 - val_accuracy: 0.3325 - lr: 0.0010
Epoch 4/10
1336/1336 [==============================] - 173s 130ms/step - loss: 2.4801 - accuracy: 0.5792 - val_loss: 3.9093 - val_accuracy: 0.3486 - lr: 0.0010
Epoch 5/10
1336/1336 [==============================] - 177s 132ms/step - loss: 2.2723 - accuracy: 0.5956 - val_loss: 3.6960 - val_accuracy: 0.3635 - lr: 0.0010
Epoch 6/10
1336/1336 [==============================] - 185s 139ms/step - loss: 2.0918 - accuracy: 0.6116 - val_loss: 3.4941 - val_accuracy: 0.3786 - lr: 0.0010
Epoch 7/10
1336/1336 [============

In [ ]:
# run for 20 more epochs
seq2seq_Model.fit(encoder_input_data, np.expand_dims(encoder_input_data, -1), epochs=20, batch_size=32, validation_split=0.2, callbacks=checkpt_list)

Epoch 1/20
1336/1336 [==============================] - 239s 169ms/step - loss: 1.5104 - accuracy: 0.6798 - val_loss: 2.9332 - val_accuracy: 0.4296 - lr: 0.0010
Epoch 2/20
1336/1336 [==============================] - 189s 142ms/step - loss: 1.4357 - accuracy: 0.6905 - val_loss: 2.8627 - val_accuracy: 0.4389 - lr: 0.0010
Epoch 3/20
1336/1336 [==============================] - 189s 142ms/step - loss: 1.3692 - accuracy: 0.7000 - val_loss: 2.8191 - val_accuracy: 0.4433 - lr: 0.0010
Epoch 4/20
1336/1336 [==============================] - 189s 142ms/step - loss: 1.3112 - accuracy: 0.7088 - val_loss: 2.7766 - val_accuracy: 0.4459 - lr: 0.0010
Epoch 5/20
1336/1336 [==============================] - 186s 139ms/step - loss: 1.2580 - accuracy: 0.7170 - val_loss: 2.7036 - val_accuracy: 0.4565 - lr: 0.0010
Epoch 6/20
1336/1336 [==============================] - 186s 139ms/step - loss: 1.2134 - accuracy: 0.7237 - val_loss: 2.6636 - val_accuracy: 0.4618 - lr: 0.0010
Epoch 7/20
1336/1336 [============

In [ ]:
seq2seq_Model.fit(encoder_input_data, np.expand_dims(encoder_input_data, -1), epochs=15, batch_size=32, validation_split=0.2, callbacks=checkpt_list)

Epoch 1/15
1336/1336 [==============================] - 198s 148ms/step - loss: 0.8590 - accuracy: 0.7854 - val_loss: 2.4054 - val_accuracy: 0.5008 - lr: 1.0000e-04
Epoch 2/15
1336/1336 [==============================] - 190s 142ms/step - loss: 0.8371 - accuracy: 0.7901 - val_loss: 2.4034 - val_accuracy: 0.5016 - lr: 1.0000e-04
Epoch 3/15
1336/1336 [==============================] - 191s 143ms/step - loss: 0.8267 - accuracy: 0.7923 - val_loss: 2.4039 - val_accuracy: 0.5023 - lr: 1.0000e-04
Epoch 4/15
1336/1336 [==============================] - 189s 141ms/step - loss: 0.8189 - accuracy: 0.7939 - val_loss: 2.4024 - val_accuracy: 0.5029 - lr: 1.0000e-04
Epoch 5/15
1336/1336 [==============================] - 193s 144ms/step - loss: 0.8120 - accuracy: 0.7953 - val_loss: 2.4037 - val_accuracy: 0.5032 - lr: 1.0000e-04
Epoch 6/15
1336/1336 [==============================] - 190s 142ms/step - loss: 0.8061 - accuracy: 0.7964 - val_loss: 2.4044 - val_accuracy: 0.5031 - lr: 1.0000e-04
Epoch 7/15

In [ ]:
seq2seq_Model.load_weights('/content/drive/MyDrive/Capstone Project MSDA/DATA 298B Code/Autoencoder_BiLSTM/weight_05.hdf5')

In [ ]:
seq2seq_Model.fit(encoder_input_data, np.expand_dims(encoder_input_data, -1), epochs=5, batch_size=32, validation_split=0.2, callbacks=checkpt_list)

Epoch 1/5
1336/1336 [==============================] - 197s 148ms/step - loss: 0.7946 - accuracy: 0.7995 - val_loss: 2.4014 - val_accuracy: 0.5037 - lr: 1.0000e-05
Epoch 2/5
1336/1336 [==============================] - 195s 146ms/step - loss: 0.7932 - accuracy: 0.7997 - val_loss: 2.4032 - val_accuracy: 0.5035 - lr: 1.0000e-05
Epoch 3/5
1336/1336 [==============================] - 193s 145ms/step - loss: 0.7923 - accuracy: 0.8000 - val_loss: 2.4044 - val_accuracy: 0.5035 - lr: 1.0000e-05


In [ ]:
seq2seq_Model.save('Autoencoder_BiLSTM.keras')

In [ ]:
## Saving Embedding portion of model
# Get the input layer and encoder layer from the trained model
input_layer = seq2seq_Model.input
encoder_layer = seq2seq_Model.layers[1].output

# Create a new model that takes the input layer and encoder layer as inputs and outputs the encoder layer's output
encoder_model = Model(inputs=input_layer, outputs=encoder_layer)

encoder_model.save('Encoder_BiLSTM.keras')

Ranking

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
bilstm_encoder = load_model('Encoder_BiLSTM.keras')

In [ ]:
bilstm_encoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder-Input (InputLayer)  [(None, 30)]              0         
                                                                 
 Body-Word-Embedding (Embed  (None, 30, 150)           300000    
 ding)                                                           
                                                                 
 bidirectional (Bidirection  (None, 256)               285696    
 al)                                                             
                                                                 
Total params: 585696 (2.23 MB)
Trainable params: 585696 (2.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(questions_answers)

In [ ]:
def new_query_processing(text):
  text_preprocess = clean_text(text)
  tokenized_input = tokenizer.texts_to_sequences([text_preprocess])
  padded_seq = pad_sequences(tokenized_input, maxlen=maxlen, padding='post')
  return padded_seq

In [ ]:
# Generate an embedding for a new question

question_text = "How do I sort a list in python?"
processed_question = new_query_processing(question_text)
question_embedding = bilstm_encoder.predict(processed_question)[0]

1/1 [==============================] - 3s 3s/step


In [ ]:
questions = np.array(ground_truth_df['Question']).astype(str)
questions.shape

(20000,)

In [ ]:
# Generate embeddings for first 10000 questions in the dataset

question_embeddings1 = []
for question in questions[:10000]:
  tokenized_input = tokenizer.texts_to_sequences([question])
  padded_seq = pad_sequences(tokenized_input, maxlen=maxlen, padding='post')
  embedding = bilstm_encoder.predict(padded_seq)[0]
  question_embeddings1.append(np.expand_dims(embedding, axis=0))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 52ms/step


In [ ]:
question_embeddings1 = np.array(question_embeddings1)
question_embeddings1.shape

(10000, 256)

In [ ]:
np.save('BiLSTM_question_embeddings_1.npy', question_embeddings1)

In [ ]:
#  next 10000 questions in the dataset

question_embeddings2 = []
for question in questions[10000:20000]:
  tokenized_input = tokenizer.texts_to_sequences([question])
  padded_seq = pad_sequences(tokenized_input, maxlen=maxlen, padding='post')
  embedding = bilstm_encoder.predict(padded_seq)[0]
  question_embeddings2.append(np.expand_dims(embedding, axis=0))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 39ms/step


In [ ]:
question_embeddings2 = np.array(question_embeddings2)
question_embeddings2.shape

(10000, 256)

In [ ]:
np.save('BiLSTM_question_embeddings_2.npy', question_embeddings2)

In [2]:
import numpy as np

question_embeddings1 = np.load('/content/drive/Shareddrives/SEM 4/DATA 298B Code/BiLSTM_question_embeddings_1.npy')
question_embeddings2 = np.load('/content/drive/Shareddrives/SEM 4/DATA 298B Code/BiLSTM_question_embeddings_2.npy')

In [4]:
tot_q_embb = np.array(list(question_embeddings1) + list(question_embeddings2))
tot_q_embb.shape 

(20000, 256)

In [5]:
np.save('BiLSTM_question_embeddings_all.npy', tot_q_embb)